In [1]:
import pandas as pd


El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en
Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso,
descripcion, estado_caso, categoria, latitud, longitud).
Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las
fechas se encuentran en el formato YYYY-MM-DD.
Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para
pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso,
respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él
El sector encargado de las estadísticas oficiales del GCPD quiere con esta información analizar las siguientes
situaciones: 

In [58]:
#Creo dataframes de prueba.

In [5]:
casos = pd.DataFrame({'fecha': ['2017-12-14', '2016-03-12', '2017-11-12', '2016-04-02', '2018-10-03', '2017-10-02', '2018-01-02', '2018-03-02', '2018-02-02', '2018-12-02', '2017-04-04', '2018-01-01', '2017-11-02'],
                        'id_caso': [0,1,2,3,4,5,6,7,8,9,10,11,12],
                        'descripcion': ['C0', 'C1', 'C2', 'C3','c4','5','6','7','8','9','10','11','12'],
                        'estado_caso': [1, 2, 1,2,1,1,1,2,1,2,3,1,1],
                        'categoria': ['A', 'D', 'C', 'D','A','C','A','D','D','A','C','A','C'],
                        'latitud': ['D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','a'],
                        'longitud': ['D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','s']},
                        index=range(0,13))

In [6]:
Batman = pd.DataFrame({'id_caso': [0,1,2,3,4,5,6,7,8,9,10,11,12],
                     'respuesta':[1,0,1,1,0,1,0,0,0,1,1,0,1]})

a) Las categorías que hayan incrementado su tasa de resolución en al menos un 10% en el último trimestre, con respecto al anterior.

In [59]:
#Me quedo con las columnas que creo necesarias

In [7]:
casosFechas = casos[['fecha','estado_caso','categoria']]

In [60]:
#Creo una columna "trimestre" segun las fechas

In [14]:
casosFechas['trimestre'] = casosFechas['fecha'].\
        apply(lambda x: "anterior" if ((x>"2018-01-00") & \
                                       (x<"2018-03-32")) else "actual")

/home/july/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
casosFechas

,fecha,estado_caso,categoria,trimestre
0,2017-12-14,1,A,actual
1,2016-03-12,2,D,actual
2,2017-11-12,1,C,actual
3,2016-04-02,2,D,actual
4,2018-10-03,1,A,actual
5,2017-10-02,1,C,actual
6,2018-01-02,1,A,anterior
7,2018-03-02,2,D,anterior
8,2018-02-02,1,D,anterior
9,2018-12-02,2,A,actual


In [62]:
#Agrupo por categoria y trimestre para poder calcular las 
#tasas de casos resueltos, pongo primero catogoria y luego trimestre
#para que cuando tenga que desapilar me quede agrupado solo por 
#categoria

In [61]:
casosGroup = casosFechas.groupby(['categoria','trimestre']).apply( \
    lambda x: x[x['estado_caso']==2].count()/x['estado_caso'].count())

In [63]:
casosGroup

fecha  estado_caso  categoria  trimestre
categoria trimestre                                          
A         actual      0.25         0.25       0.25       0.25
          anterior    0.00         0.00       0.00       0.00
C         actual      0.00         0.00       0.00       0.00
D         actual      1.00         1.00       1.00       1.00
          anterior    0.50         0.50       0.50       0.50

In [64]:
#Me quedo con una sola de las columnas que hara de tasa y la renombro

In [21]:
casosA = casosGroup[['fecha']].rename(columns={'fecha':'tasa'})

In [22]:
casosA

tasa
categoria trimestre      
A         actual     0.25
          anterior   0.00
C         actual     0.00
D         actual     1.00
          anterior   0.50

In [65]:
#Desapilo para tener en columnas los datos de cada trimestre

In [23]:
tasaCasos = casosA.unstack()

In [24]:
tasaCasos

tasa         
trimestre actual anterior
categoria                
A           0.25      0.0
C           0.00      NaN
D           1.00      0.5

In [66]:
#Me quedo con la tasa

In [36]:
tasas = tasaCasos['tasa']

In [37]:
tasas

trimestre,actual,anterior
categoria,,
A,0.25,0.0
C,0.00,NaN
D,1.00,0.5


In [67]:
#Me quedo con las categorias en donde se haya superado 
#incrementado la tasa como se pide en el enunciado

In [40]:
cate = tasas[tasas['actual']>(tasas['anterior']+tasas['anterior']*0.1)]

In [41]:
cate

trimestre,actual,anterior
categoria,,
A,0.25,0.0
D,1.00,0.5


b) Tasa de participacion de batman por categoria,para los delitos contra la propiedad (que enmarcan las categorias incendio internacional, robo, hurto y robo de vehiculos)

In [68]:
#Me quedo con las columndas de interes

In [43]:
casosCategoria = casos[['id_caso','categoria']]

In [69]:
#Me quedo con las categorias, en este caos A y D simulan las que piden

In [70]:
casosCategoria = casosCategoria[(casosCategoria['categoria']=='A')|\
                                (casosCategoria['categoria']=='D')]

In [ ]:
#Hago un merge entre casos y batman, observar que se hace por id_caso
#y a traves de un left mergem esto es asi porque  me interesan los 
#casos totales sin importar que se haya llamado o no a batman, ya que 
#la tasa de participacion se calcula sobre los casos totales en ciudad 
#gotica.

In [71]:
casosBatiseñal = pd.merge(casosCategoria,Batman, on= 'id_caso',\
                          how='left')

In [48]:
casosBatiseñal

,id_caso,categoria,respuesta
0,0,A,1
1,1,D,0
2,3,D,1
3,4,A,0
4,6,A,0
5,7,D,0
6,8,D,0
7,9,A,1
8,11,A,0


In [ ]:
#Hago un Fillna para filtrar, en el caso de que los hubiera habido, valores nulos,
#es decir casos de la fuerza polical donde no se activo la batiseñal,
#este valor nulo se reemplaza por cero indicanfo que no hubo respuesta
#de batman

In [51]:
casosBatiseñal = casosBatiseñal.fillna(value={'respuesta':0})

In [ ]:
#calculo las tasas por categoria

In [54]:
casosBatiseñal = casosBatiseñal.groupby('categoria').apply(lambda x:\
                    x[x['respuesta']==1].count()/x['respuesta'].count())

In [55]:
casosBatiseñal

,id_caso,categoria,respuesta
categoria,,,
A,0.40,0.40,0.40
D,0.25,0.25,0.25


In [72]:
#Me quedo con una columna que hara de tasa

In [56]:
tasa = casosBatiseñal[['respuesta']].rename(columns={'respuesta':'tasa'})

In [57]:
tasa

,tasa
categoria,
A,0.40
D,0.25
